In [1]:
from sanic import Sanic
import uuid

import logging
import json


In [2]:
LOG_FORMAT = "%(asctime)s [%(levelname)s]: %(threadName)s - %(message)s"
logging.basicConfig(format=LOG_FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel("INFO")

[x] Cada jogador inicia com 10 cartas;

[ ] ~~ Toda rodada é sorteado um jogador para receber três cartas pretas e escolher qual vai ser a carta da rodada;~~

[ ] O vencedor da rodada escolherá a carta de pergunta da rodada seguinte

[ ] ~~O jogo termina quando acabarem as cartas brancas;~~

Algumas cartas têm 2 espaços pra resposta, já está mapeado no bd;

Para cartas com 2 respostas, todos jogadores recebem +1 carta;

Regra RANDO: Toda rodada o jogador dummy recebe uma carta, caso a dele seja votada mais engraçada, o jogo acaba e todos perdem;

No fim da rodada, cada jogador vota em uma carta; A mais votada recebe um ponto; Ganha quem chegar no máximo de pontuação mais rápido;

A pontuação é definida no início da partida, ao criar a sala;

Modo sobrevivência: Após o fim das respostas, todos os jogadores votam em uma carta para ser apagada, a última restante ganha;

É possível sacrificar 1 ponto para girar suas cartas em mãos;

A cada rodada, o jogador pode puxar mais uma carta;

Cartas brancas não repetem no deck;

In [4]:
class Resposta():
  def __init__(self, texto: str, autor:str = 'Cartas contra a humanidade') -> None:
    self.texto = texto
    self.ingame = False
    self.autor = autor
    self.jogador = None
    self.votos = 0
    self.id = uuid.uuid4()
  
  def marcar_ingame(self):
    self.ingame = True

  def to_json(self) -> str:
    obj = {
      'texto': self.texto,
      'autor': self.autor,
      'id': self.id
    }

    return json.dumps(obj)

  def debug(self):
    return f"""Texto: {self.texto}\nIngame: {self.ingame}\nVotos: {self.votos}\nID: {self.id}\nAutor: {self.autor}\nJogador: {self.jogador}"""

In [93]:
class Jogador:
  def __init__(self, nome: str) -> None:
    self.nome = nome
    self.cartas = []
    self.pontuacao = 0
    self.status = 'aguardando' # possíveis status = aguardando, escolhendo, votando
    self.id = uuid.uuid4()

    logger.info('Jogador %s foi criado com o id %s.', self.nome, self.id)

  def descartar_resposta(self, resposta: Resposta):
    self.cartas = [carta for carta in self.cartas if str(carta.id) != str(resposta.id)]
    logger.info("A carta de ID %s foi descartada do deck do jogador %s", str(resposta.id), self.id)

  def to_json(self) -> str:
    response = {
      "nome": self.nome,
      "id": self.id,
    }

    return response

  def debug(self) -> None:
    print(str(self))

  def __str__(self) -> str:
    txt = f"""\n\tNome: {self.nome}\n\tCartas: {','.join([carta.texto for carta in self.cartas])}\n\tPontuação: {self.pontuacao}\n\tStatus: {self.status}\n\tID: {self.id}"""

    return txt
    

In [94]:
class Pergunta():
  def __init__(self, texto: str, autor: str = 'default') -> None:
    self.texto = texto
    self.ingame = False
    self.picks = max(texto.count('_'), 1)
    self.autor = autor
    self.id = uuid.uuid4()

  def to_json(self) -> str:
    obj = {
      'texto': self.texto,
      'picks': self.picks,
      'autor': self.autor,
      'id': self.id
    }

    return json.dumps(obj)

  def debug(self):
    return f"""Texto: {self.texto}\nIngame: {self.ingame}\nPicks: {self.picks}\nID: {self.id}\nAutor: {self.autor}"""

In [95]:
from random import choice

class Deck:
  """ ### Responsabilidades do Deck

Ler as cartas da memória

Selecionar as cartas para serem escolhidas como pergunta

Distribuir as cartas para os jogadores

Marcar cartas como ingame (Perguntas e Respostas)

  """
  def __init__(self) -> None:
    self.perguntas = []
    self.respostas = []

    logger.info('Deck foi iniciado.')
    self.carregar_cartas()

  def carregar_cartas(self):
    """Faz a leitura das cartas no banco de dados
    """
    with open('/home/costa/Projetos/cards_against_the_humanity/server/src/objects/perguntas.txt') as f:
      for pergunta in f.readlines():
        self.perguntas.append(Pergunta(pergunta[:-2]))

      logger.info('Foram carregadas %s perguntas', len(self.perguntas))  
      
    
    with open('/home/costa/Projetos/cards_against_the_humanity/server/src/objects/respostas.txt') as f:
      for resposta in f.readlines():
        self.respostas.append(Resposta(resposta[:-2]))
      logger.info('Foram carregadas %s respostas', len(self.respostas))  

  def pick_respostas(self, k: int = 10):
    """Seleciona 10 cartas arbitrárias entre as cartas que ainda não estão em jogo.
    Automaticamente as marca como ingame.

    Args:
        k (int, optional): Quantidade de cartas a serem puxadas. Defaults to 10.

    Raises:
        Exception: Caso acabem as cartas disponíveis, levanta um erro

    Returns:
        list(Resposta): Lista com as 10 respostas já marcadas como ingame
    """
    respostas_nao_usadas = [resposta for resposta in self.respostas if resposta.ingame == False ]
    logger.info(f"respostas restantes: {len(respostas_nao_usadas) - 10}")

    if(len(respostas_nao_usadas) == 0):
      raise Exception('Acabaram as cartas, porra')

    respostas = [choice(respostas_nao_usadas) for i in range(k)]

    for resposta in respostas:
      resposta.marcar_ingame()

    if k == 1:
      # Caso onde é a primeira carta a ser jogada
      # não passa pelo processo de seleção dos jogadores
      self.marcar_resposta_ingame(respostas[0].id)

    return respostas

  def pick_perguntas(self, k: int = 3) -> list[Pergunta]:
    """Escolhe aleatoriamente 3 perguntas da lista de perguntas ainda não escolhidas

    Params:
        k (int): Quantidade de perguntas a serem retornadas

    Raises:
        Exception: Erro caso não hajam mais cartas para serem escolhidas

    Returns:
        list[Pergunta]: Lista com as 3 perguntas para o jogador escolher.
    """
    perguntas_nao_usadas = [pergunta for pergunta in self.perguntas if pergunta.ingame == False ]

    logger.info(f"Perguntas restantes: {len(perguntas_nao_usadas)}")

    if(len(perguntas_nao_usadas) == 0):
      raise Exception('Acabaram as cartas, porra')

    perguntas = [choice(perguntas_nao_usadas) for i in range(k)]

    if k == 1:
      # Caso onde é a primeira carta a ser jogada
      # não passa pelo processo de seleção dos jogadores
      self.marcar_pergunta_ingame(perguntas[0].id)

    return perguntas
  
  def marcar_pergunta_ingame(self, id):
    """Recebe um ID de pergunta e marca ela como já utilizada em jogo

    Args:
        id (str): identificador da carta a ser marcada

    Raises:
        Exception: Exception de not found
    """
    pergunta = [pergunta for pergunta in self.perguntas if pergunta.id == id].pop()
    if not pergunta:
      raise Exception('Pergunta não existe na lista de perguntas')
    pergunta.ingame = True
    
    logger.info("%s", pergunta.debug())

# Passo a passo da partida

[x] Criar a partida com os parâmetros de jogo

[x] Esperar todos os jogadores entrarem

[x] Iniciar o jogo (apenas o dono)

[x] Distribuir as cartas para cada jogador

[x] Selecionar a carga de pergunta da rodada (apenas a primeira rodada)

[ ] Cada jogador recebe sua carta

[ ] Cada jogador pode puxar uma carta nova 1x por rodada

[ ] Votação

[ ] Vencedor escolhe a carta pergunta das próximas rodadas

[ ] Repete o jogo em ciclo até a pontuação máxima chegar ou o máximo de rodadas

In [139]:
class Mesa:

  def __init__(self, deck: Deck) -> None:
    self.deck = deck
    
    self.pergunta_da_rodada = None
    self.respostas_da_rodada = []
    logger.info('Mesa foi iniciada.')

  def reiniciar(self):
    self.pergunta_da_rodada = []
    self.respostas_da_rodada = []

  def adicionar_resposta(self, resposta: Resposta):
    logger.info("A resposta '%s' foi adicionada à mesa e marcada como usada ingame", resposta.texto)
    self.respostas_da_rodada.append(resposta)

  def debug(self):
    return f"Pergunta da rodada: \n\tTexto: {self.pergunta_da_rodada.texto}\n\tPicks: {self.pergunta_da_rodada.picks}\nRespostas em jogo: {[resposta.texto for resposta in self.respostas_da_rodada]}"


In [202]:
class Partida:
    """
    Entidade da partida
    """
    def __init__(
        self,
        jogador: Jogador,
        max_pontuacao: int = 10,
        temporizador_votacao: int = 30,
        temporizador_selecao: int = 30,
        max_rodadas: int = 15,
    ) -> None:

        # Valores devault
        self.temporizador_votacao = temporizador_votacao
        self.temporizador_selecao = temporizador_selecao
        self.max_pontuacao = max_pontuacao
        self.max_rodadas = max_rodadas
        self.dono = jogador

        self.jogadores = [self.dono]
        self.rodada_atual = 0
        self.deck = Deck()
        self.mesa = Mesa(deck=self.deck)
        self.id = uuid.uuid4()
        self.started = False

        logger.info('A entidade da Partida foi criada com o ID %s', self.id)

    def __str__(self) -> str:
        debug_text = f"""ID: {self.id}\nStarted: {self.started}\nTemporizador de seleção: {self.temporizador_selecao}\nTemporizador de votação: {self.temporizador_votacao}\nPontuação máxima: {self.max_pontuacao}\nMáximo de rodadas: {self.max_rodadas}\nJogadores: {len(self.jogadores)}\nDono da sala: {str(self.dono)}\n"""
        return debug_text

    def iniciar_jogo(self, id: str) -> str:
        try:
            if id == str(self.dono.id):
                self.started = True
                self.gerar_primeira_pergunta()

                for jogador in self.jogadores:
                    jogador.cartas.extend(self.deck.pick_respostas())
                    
                    for carta in jogador.cartas:
                        carta.jogador = jogador.nome

                return 'OK'
            else:
                return 'NOT STARTED'
            
        except Exception as e:
            logger.exception(e)

    
    def gerar_primeira_pergunta(self) -> None:
        pergunta_inicial = self.deck.pick_perguntas(1)
        self.mesa.pergunta_da_rodada = pergunta_inicial[0]

    def gerar_perguntas(self):
        perguntas = self.deck.pick_perguntas()
        return perguntas

    def selecionar_pergunta(self, payload: dict):

        jogador = self.get_jogador(payload['jogador'])
        
        if len(payload['resposta']) > int(self.mesa.pergunta_da_rodada.picks):
            logger.info("O jogador %s enviou mais respostas do que o necessário.", jogador.nome)
            raise Exception("Mais respostas do que deveria.")

        respostas_da_rodada = [str(resposta.jogador) for resposta in self.mesa.respostas_da_rodada]

        if jogador.nome in respostas_da_rodada:
            raise Exception("Jogador já fez sua seleção.")

        logger.info("Jogadores que já responderam a rodada: %s", respostas_da_rodada)

        for resposta in payload['resposta']:
            resposta = self.get_resposta(resposta)

            try:
                self.mesa.adicionar_resposta(resposta)
                jogador.descartar_resposta(resposta)
            except Exception as e:
                logger.exception(e)
                raise Exception(e)

    def entrar_na_sala(self, novo_jogador: Jogador) -> uuid.UUID:
        try:
            if novo_jogador.id not in [jogador.id for jogador in self.jogadores]:
                self.jogadores.append(novo_jogador)
            else:
                raise Exception('Jogador de mesmo ID já está no lobby')
            
            logger.info("O jogador %s entrou na sala com o ID %s", novo_jogador.nome, novo_jogador.id)
            return self.id
        except Exception as e:
            logger.exception(e)
        

    def get_resposta(self, id_resposta: str):
        resposta = [resposta for resposta in self.deck.respostas if str(resposta.id) == id_resposta]
        if len(resposta) > 0:
            return resposta[0]

    def get_jogador(self, id_jogador: str):
        jogador = [jogador for jogador in self.jogadores if str(jogador.id) == id_jogador]
        
        if len(jogador) > 0:
            return jogador[0]


    def debug_jogadores(self) -> None:
        for jogador in self.jogadores:
            print(str(jogador))

    def debug(self) -> None:
        print(str(self))


In [203]:
dummy = Jogador('isaac')
dummy_2 = Jogador('vitu')

game = Partida(dummy, 10, 30, 15)
game.entrar_na_sala(dummy_2)

2024-04-28 04:07:38,213 [INFO]: MainThread - Jogador isaac foi criado com o id b1f23005-47ee-4287-bb3b-7547180b27cb.
2024-04-28 04:07:38,214 [INFO]: MainThread - Jogador vitu foi criado com o id 9dcfa90a-886b-4afd-8b20-b373447d563f.
2024-04-28 04:07:38,214 [INFO]: MainThread - Deck foi iniciado.
2024-04-28 04:07:38,214 [INFO]: MainThread - Foram carregadas 95 perguntas
2024-04-28 04:07:38,215 [INFO]: MainThread - Foram carregadas 403 respostas
2024-04-28 04:07:38,216 [INFO]: MainThread - Mesa foi iniciada.
2024-04-28 04:07:38,216 [INFO]: MainThread - A entidade da Partida foi criada com o ID 368da1cc-bca6-4ff0-b265-11239d9f2d66
2024-04-28 04:07:38,216 [INFO]: MainThread - O jogador vitu entrou na sala com o ID 9dcfa90a-886b-4afd-8b20-b373447d563f


UUID('368da1cc-bca6-4ff0-b265-11239d9f2d66')

In [204]:
game.iniciar_jogo(str(dummy.id))

2024-04-28 04:07:38,487 [INFO]: MainThread - Perguntas restantes: 95
2024-04-28 04:07:38,487 [INFO]: MainThread - Texto: Eu bebo para esquecer _.
Ingame: True
Picks: 1
ID: d229ac5a-d243-4377-823d-334a5e7b2e89
Autor: default
2024-04-28 04:07:38,487 [INFO]: MainThread - respostas restantes: 393
2024-04-28 04:07:38,488 [INFO]: MainThread - respostas restantes: 383


'OK'

In [205]:
game.debug_jogadores()


	Nome: isaac
	Cartas: Ciências,Um filhote de crocodilo mordendo a cabeça do seu pênis,A placenta,Assistir Cartoon Network pelado(a),Pedófilos,Cagar para o Terceiro Mundo,A fome,Coceira no toba,Limpeza étnica,Mendigos
	Pontuação: 0
	Status: aguardando
	ID: b1f23005-47ee-4287-bb3b-7547180b27cb

	Nome: vitu
	Cartas: Ejaculação precoce,Fazer uma lobotomia,Nem mole, nem duro,Uma diarreia brutal,Uma mulata amputada,Minha vida sexual,Apertar um baseado,Gente pobre,Uma bactéria carnívora,Os dinossauros
	Pontuação: 0
	Status: aguardando
	ID: 9dcfa90a-886b-4afd-8b20-b373447d563f


In [206]:
for jogador in game.jogadores:
  print('Jogador: {}\nID: {}'.format( jogador.nome.capitalize(), jogador.id))
  
  for carta in jogador.cartas:
    print(carta.id)

Jogador: Isaac
ID: b1f23005-47ee-4287-bb3b-7547180b27cb
758ffecb-8794-48e4-9d5f-d7416dc08671
4e7d38e2-7e3b-4d6d-9ec3-b1f47e0b79c5
b146f30c-a562-49b4-8416-3946d65cc65b
753b2856-745f-40dd-8cbe-360ad1b7f2e3
ea1a5635-0c97-4740-8eec-cf7d2c6977e6
0fbf98cd-9642-4435-bdf3-219a4e0222f8
3e1d75b0-e071-4832-8948-5776aa4d7a09
2d88de09-cf59-4bed-a0b7-a485f6b713c6
7e569f14-0e94-4a81-bae2-3f2d5599afb6
d9dc9674-a426-4478-9d8a-26860c8c153c
Jogador: Vitu
ID: 9dcfa90a-886b-4afd-8b20-b373447d563f
ac4166b1-41c5-4bb4-9032-cf5db5892b01
69d261b4-3e69-496c-9394-fb0d6210d1e5
6cd11264-51d1-4c11-b886-3d50426c7cff
293198de-0346-4e1d-8460-d514ee1eb36f
2e80f99c-f507-42db-9baf-7903b89454c6
eab5c2ae-08b2-4c4b-9e50-2cdf3cba1cb4
2602d484-d18e-4010-ab20-ddd35da874e5
d64a8d71-822c-4c19-ab06-59df75c89eee
8bd2a7d7-6dd1-4237-87e0-9a09bbccf628
7cf0733f-74aa-4f0f-9c52-daf5a3fa06d1


In [207]:
# Simulação de carta jogada

payload = {
  'jogador': str(dummy.id),
  'resposta': [str(dummy.cartas[0].id)]
}

In [210]:
game.selecionar_pergunta(payload=payload)

Exception: Jogador já fez sua seleção.

In [211]:
print(game.mesa.debug())

Pergunta da rodada: 
	Texto: Eu bebo para esquecer _.
	Picks: 1
Respostas em jogo: ['Ciências']


In [212]:
[jogador for jogador in game.jogadores if str(jogador.id) == payload['jogador']]